In [1]:
%config ZMQInteractiveShell.ast_node_interactivity='all'
%matplotlib inline
import warnings;warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
import os
import matplotlib.pyplot as plt

In [2]:
############## Import

d = pd.read_csv('./data/application_train.csv')
d.shape
d.head()

####### Data Exploration
d_info = {}

(307511, 122)

SK_ID_CURR  TARGET NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR  \
0      100002       1         Cash loans           M            N   
1      100003       0         Cash loans           F            N   
2      100004       0    Revolving loans           M            Y   
3      100006       0         Cash loans           F            N   
4      100007       0         Cash loans           M            N   

  FLAG_OWN_REALTY  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  \
0               Y             0          202500.0    406597.5      24700.5   
1               N             0          270000.0   1293502.5      35698.5   
2               Y             0           67500.0    135000.0       6750.0   
3               Y             0          135000.0    312682.5      29686.5   
4               Y             0          121500.0    513000.0      21865.5   

   ...  FLAG_DOCUMENT_18 FLAG_DOCUMENT_19 FLAG_DOCUMENT_20 FLAG_DOCUMENT_21  \
0  ...                 0                0                0                0   
1  ...                 0                0                0                0   
2  ...                 0                0                0                0   
3  ...                 0                0                0                0   
4  ...                 0                0                0                0   

  AMT_REQ_CREDIT_BUREAU_HOUR AMT_REQ_CREDIT_BUREAU_DAY  \
0                        0.0                       0.0   
1                        0.0                       0.0   
2                        0.0                       0.0   
3                        NaN                       NaN   
4                        0.0                       0.0   

   AMT_REQ_CREDIT_BUREAU_WEEK  AMT_REQ_CREDIT_BUREAU_MON  \
0                         0.0                        0.0   
1                         0.0                        0.0   
2                         0.0                        0.0   
3                         NaN                        NaN   
4                         0.0                        0.0   

   AMT_REQ_CREDIT_BUREAU_QRT  AMT_REQ_CREDIT_BUREAU_YEAR  
0                        0.0                         1.0  
1                        0.0                         0.0  
2                        0.0                         0.0  
3                        NaN                         NaN  
4                        0.0                         0.0  

[5 rows x 122 columns]

In [8]:
####### Target Distribution

target = 'TARGET'

d[target].mean()
d[target].value_counts()

0.08072881945686496

0    282686
1     24825
Name: TARGET, dtype: int64

In [19]:
####### Coverage

def calc_coverage(data):
    scale = 2
    coverages = round(1 - data.isnull().sum(axis=0)/len(data), scale)
    r = coverages.sort_values(ascending=False)
    return r

d_info['coverage'] = calc_coverage(d)
d_info['coverage'].describe()

count    122.000000
mean       0.756393
std        0.283945
min        0.300000
25%        0.490000
50%        1.000000
75%        1.000000
max        1.000000
dtype: float64

In [45]:
############## Data types

types = d.dtypes

'''*** Data Types ***'''
types.value_counts()

'''*** String Columns ***'''

def ana_dtypes(t):
    if t == 'str':
        cols = types[types == 'object'].index
    else:
        cols = types[types != 'object'].index
    # Save description
    cols_desc = d[cols].describe().T
    cols_desc.to_csv(f'./data/cols_desc_{t}.csv')
    # Return columns
    return cols

columns = {}
for t in 'str', 'num':
    columns[t] = ana_dtypes(t)

'*** Data Types ***'

float64    65
int64      41
object     16
dtype: int64

'*** String Columns ***'

In [28]:
############## Binarize二值化

a = d['CNT_FAM_MEMBERS'].fillna(0) # 不能为空
bn = preprocessing.Binarizer(threshold=10) # >threshold为1. 默认threshold=0;
b = bn.transform([a]) # input需要是二维

NAME_CONTRACT_TYPE    object
dtype: object

In [29]:
############## Segmentation分段

# 等深
pd.qcut(data[pred], 10, labels=False, duplicates='drop')
# 等宽
pd.cut()
# 不同数据取相同区间
bins[x0] = pd.qcut(data[x0], 10).values.categories
data[f'{x0}_bin'] = bins[x0].get_indexer(data[x0].round(3))

Cash loans         278232
Revolving loans     29279
Name: NAME_CONTRACT_TYPE, dtype: int64

In [ ]:
############## boxcox

from scipy import stats as spstats
l, opt_lambda = spstats.boxcox(a)

In [ ]:
############## outlier

# http://www.itkeyword.com/doc/5804810393939396x299/remove-outliers-in-pandas-dataframe-using-percentiles
low = .05
high = .95
quant_df = filt_df.quantile([low, high])

filt_df = filt_df.apply(lambda x: x[(x>quant_df.loc[low,x.name]) & 
                                    (x < quant_df.loc[high,x.name])], axis=0)